In [24]:
import pandas as pd
import numpy as nb
from config import *
import json
import re

In [209]:
scenario_file_name = "generated_scenarios_20250922_1207"

In [210]:
output_file = path_results + scenario_file_name + ".jsonl"

In [211]:
df_crh= pd.read_csv(path_results + "crh_"+scenario_file_name + ".csv",index_col=0)

In [213]:
limit = 50
with open(output_file, 'a+', encoding="utf-8") as outfile:

    for i in range(0,len(df_crh)) :

        if  len(df_crh.crh[i])<500 :

            continue
        # annotations = json.loads(df_crh.crh[i].split('"formulations":')[1][:-3])
        case_management_type = df_crh.case_management_type[i]
        case_management_type_description = df_crh.case_management_type_description[i]

        diagnostics = df_crh.crh[i].split('"formulations":')[1][:-3].replace('\n ',"").split("informations")[0][:-5].split('"diagnostics":')[1]

        text_crh = df_crh.crh[i].split('"formulations":')[0].split('"CR":')[1][3:-5]

        codes = []
        #Regular expressions 
        p = re.compile("^[A-Z][0-9]{2}")
        PCID = "Diagnostic principal :\n"
        SICD = "Diagnotics associés :\n"
        CMT = "Motif de recours au soin (code en Z du chapitre XXI):\n"

        if case_management_type == "DP":
            CMT+= "- aucun, le motif de recours au soin est le diagnostic principal\n"
        else :
            CMT+= "- " + case_management_type_description + "(" + case_management_type + ")\n"

        for x in re.sub('"',"", diagnostics).split("],"):
            diag =  re.sub("\[|\]|}|{","",x).replace("  ","")
            extract = re.findall(r'\((.*?)\)',diag)
            if extract:
                for code in extract: 
                    if p.match(code):
                        codes +=[code]
                        if(code == df_crh.icd_primary_code.loc[i]):
                            PCID += "- " + diag + "\n"
                        else :
                            SICD += "- " + diag + "\n"
        
                    
            
        data = {
        "messages": [
            {
                "role": "system",
                "content": """Vous êtes un modèle de langage en française spécialisé dans le codage des diagnostics selon la classification internationale des maladies version 10 (CIM-10) pour les résumés standardisés de sortie du programme de médicalisation des systèmes d'information français (PMSI). A partir des comptes rendus d'hospitalisation vous donnerez  les codes diagnostics CIM-10 que l'on peut retenir pour le séjours en distiguant  :
- Motifs de recours (code en Z du chapitre XXI). Ces codes ne sont utilisables que dans certaines circonstances. Lorsqu'il n'est pas nécessaire vous d'utiliser un code en Z vous votre réponse pour cette catégorie sera : aucun, le motif de recours au soin est le diagnostic principal.
- Diagnostic principal 
- Diagnostics associés """},

            {
                "role": "user",
                "content": "Générez le codage CIM-10 du résumé strandisé de sortie PMSI à partir du compte rendu d'hospitalisation suivant : " + text_crh
            },
            {
                "role": "assistant",
                "content": "Codes CIM 10 retenus pour le résumé strandisé de sortie PMSI : " + CMT+PCID+SICD
            }
        ]
        }
        json.dump(data, outfile, separators=(',', ':'),  ensure_ascii=False)
        outfile.write('\n')


        if limit is not None:
            if i > limit:
                break

In [220]:
print(CMT+PCID+SICD)

Motif de recours au soin (code en Z du chapitre XXI):
- aucun, le motif de recours au soin est le diagnostic principal
Diagnostic principal :
- Travail et accouchement compliqués d'une anomalie du rythme cardiaque du foetus (O680):anomalies du rythme cardiaque fœtal, décélérations variables du RCF, tracé de type II, signes de détresse fœtale chronique, anomalies du RCF persistantes 
Diagnotics associés :
-  Centres médicaux non disponibles et non accessibles (Z753):difficultés d’accès aux soins, suivi obstétrical irrégulier, éloignement géographique, précarité sociale 
-  Maladies à protozoaires compliquant la grossesse, l'accouchement et la puerpéralité (O986):paludisme traité en cours de grossesse, antécédent de paludisme à *Plasmodium falciparum*, sérologie paludéenne positive au 2ᵉ trimestre, contexte infectieux antérieur (paludisme) 
-  Rupture d'une suture de césarienne (O900):risque de désunion de cicatrice de césarienne// Mentionné indirectement dans la surveillance post-op 
- 

In [214]:
from mistralai import Mistral
import os


client = Mistral(api_key=api_key)

file_1 = client.files.upload(file={
    "file_name": scenario_file_name + ".jsonl",
    "content": open(output_file, "rb"),
})


In [217]:
scenario_file_name = "generated_scenarios_20250922_1208"
output_file = path_results + scenario_file_name + ".jsonl"

In [218]:
file_2 = client.files.upload(file={
    "file_name": scenario_file_name + ".jsonl",
    "content": open(output_file, "rb"),
})


In [219]:
# create a fine-tuning job
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b", 
    training_files=[{"file_id": file_1.id, "weight": 1}],
    validation_files=[file_2.id], 
    hyperparameters={
        "training_steps": 10,
        "learning_rate":0.0001
    },
    auto_start=False
)

# start a fine-tuning job
client.fine_tuning.jobs.start(job_id = created_jobs.id)

created_jobs

SDKError: API error occurred: Status 400
{"detail": "Job has status QUEUED, cannot start unless job has status VALIDATED."}

In [208]:
print(created_jobs)

id='c5c72831-0bf3-42ea-b61f-f5c8aa323d00' auto_start=False hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None, seq_len=32768) model='open-mistral-7b' status='QUEUED' job_type='completion' created_at=1758724866 modified_at=1758724866 training_files=['b423ac5d-3f17-473f-a667-2234c4ffbc4e'] validation_files=['89ce81ce-d885-446e-bd23-48284e872ccf'] OBJECT='job' fine_tuned_model=None suffix=None integrations=[] trained_tokens=None repositories=[] metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None)
